In [ ]:
#UPVOTE IF YOU FIND IT USEFUL :D
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
#reading CSV with pandas
Taxonomy = pd.read_csv('../input/birdclef-2022/eBird_Taxonomy_v2021.csv')
test = pd.read_csv('../input/birdclef-2022/test.csv')
train_metadata = pd.read_csv('../input/birdclef-2022/train_metadata.csv')
submission = pd.read_csv('../input/birdclef-2022/sample_submission.csv')
from os import path
from pydub import AudioSegment
import seaborn as sns
from scipy.io.wavfile import read
from statsmodels.graphics.tsaplots import plot_acf
from scipy import signal
from scipy.io import wavfile


#transform into dataframe
df_taxonomy = pd.DataFrame(Taxonomy)

df_test = pd.DataFrame(test)

df_train = pd.DataFrame(train_metadata)

df_submission = pd.DataFrame(submission)









In [ ]:
#opening the scored bird file
with open('../input/birdclef-2022/scored_birds.json','r') as f:
    data = json.load(f)
    

In [ ]:
#index of the scored rows
index_row_scored = []
for i in data:
    a=i.strip('"')
    for j in range(len(df_train["primary_label"])):
        if df_train["primary_label"][j] == a:
            index_row_scored.append(j)


In [ ]:

from os import listdir
from os.path import isfile, join
rated_list = []
for i in data:
    onlyfiles = [f for f in listdir("../input/birdclef-2022/train_audio/"+i) if isfile(join("../input/birdclef-2022/train_audio/"+i,f))]
    rated_list.append(i)
    rated_list.append(onlyfiles)
    

In [ ]:
#Function to extract features from the audios
def features_extractor(file):
    #load the file (audio)
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    #we extract mfcc
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    #in order to find out scaled feature we do mean of transpose of value
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features


In [ ]:
#We will use a dataframe with the scored birds just to see that everything works fine 
#without spending so much time
df_scored = df_train.copy()
for i in df_train.index:
    #if df_train['rating'][i]<2.5:
        #df_train.drop([i],axis = 0, inplace = True)
    if i not in index_row_scored:
        df_scored.loc[i,'primary_label'] = 'other'

In [ ]:
df_scored

In [ ]:

from tqdm import tqdm
# Now we iterate through every audio file and extract features 
# using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(df_scored.iterrows()):
    file_name = os.path.join(os.path.abspath("../input/birdclef-2022/train_audio/"),str(row["filename"]))
    final_class_labels=row["primary_label"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','primary_label'])
extracted_features_df.head()

In [ ]:
#Now we split the data into train and test
from imblearn.over_sampling import RandomOverSampler #we improved accuracy with this!
oversample = RandomOverSampler(sampling_strategy='minority')
# Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['primary_label'].tolist())
# Label Encoding -> Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
### Train Test Split
from sklearn.model_selection import train_test_split
X,y = oversample.fit_resample(X, y)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.02)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
trainedforest = RandomForestClassifier(n_estimators = 700).fit(X_train, y_train)
predictionforest = trainedforest.predict(X_test)
print(classification_report(y_test,predictionforest))

In [ ]:
#we create feature names
#X=np.array(extracted_features_df['feature'].tolist())
X_t = []
X__t = []
for j in range(0,40):
    X_t = []
    for i in range(len(X)):
        X_t.append(X[i][j])
    X__t.append(X_t)
    
feature_names = [f"feature {i}" for i in range(X.shape[1])]


In [ ]:
#Feature importance
from matplotlib.pyplot import figure
figure(num = None, figsize=(10,11), dpi = 80, facecolor = 'w', edgecolor = 'k')
feat_importances = pd.Series(trainedforest.feature_importances_, index = feature_names)
feat_importances.nlargest(40).plot(kind = 'barh')

In [ ]:
#We use 7 features

from matplotlib.pyplot import figure
figure(num = None, figsize=(10,11), dpi = 80, facecolor = 'w', edgecolor = 'k')
feat_importances = pd.Series(trainedforest.feature_importances_, index = feature_names)
feat_importances.nlargest(7).plot(kind = 'barh')

In [ ]:
#We select the 7 features with more importance in our data
X_prueba = X
X_prueb = X_prueba.tolist()
X_pru = []
X_pri = []

for i in range(len(X)):
    for j in range(0,40):
        if j==6 or j==8 or j==2 or j==16 or j==12 or j==24 or j==36:
            X_pru.append(X_prueb[i][j])
    X_pri.append(X_pru)
    X_pru = []
            
                
X_pro = np.array([np.array(xi) for xi in X_pri])
print(X_pro)
len(X_pro)

In [ ]:
#wedivide again between train and test
#X_train,X_test,y_train,y_test=train_test_split(X_pro,y,test_size=0.2)
#X_train.shape
#X_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
# No of classes
num_labels=y.shape[1]

In [ ]:
#Creation of the model
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(7,)))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
num_epochs = 200
num_batch_size = 32
checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()
model.fit(X_pro, y, batch_size=num_batch_size, epochs=num_epochs)
duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:

# Let's check the accuracy!
#test_accuracy=model.evaluate(X_test,y_test,verbose=0)
#print(test_accuracy[1])

In [ ]:
#predict_x = model.predict(X_test)
#classes_x = np.argmax(predict_x, axis = 1)
#predict_x[1]

In [ ]:
#Here we split the test audio file
test_audio_dir = '../input/birdclef-2022/test_soundscapes/'
file_list = [f.split('.')[0] for f in sorted(os.listdir(test_audio_dir))]

In [ ]:
with open('../input/birdclef-2022/scored_birds.json') as sbfile:
    scored_birds = json.load(sbfile)

In [ ]:
pred = {'row_id': [], 'target': []}

In [ ]:
# Process audio files and make predictions
for afile in file_list:
    
    # Complete file path
    path = test_audio_dir + afile + '.ogg'
    
    # Open file with librosa and split signal into 5-second chunks
    # sig, rate = librosa.load(path)
    # ...
    
    # Let's assume we have a list of 12 audio chunks (1min / 5s == 12 segments)
    chunks = [[] for i in range(12)]

In [ ]:
chunks

In [ ]:
#y_test.shape
#predict_x.shape
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
#score = f1_score(str(y_test[1]),str(predict_x[1]).replace(".", ""), average = None)
#fil_acc_orig = accuracy_score(X_test,predict_x)

#X_test

#y



Let's prepare the test file to create the submision csv

In [ ]:
# we will divide the test audio in 12 audios of 5 seconds.

from pydub import AudioSegment
from pydub.utils import make_chunks 
sound = AudioSegment.from_file("../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg")

chunk_length_ms = 5000 # pydub calculates in millisec 
chunks = make_chunks(sound,chunk_length_ms) #Make chunks of one sec 
for i, chunk in enumerate(chunks): 
    chunk_name = "{0}.ogg".format(i) 
    print ("exporting", chunk_name) 
    chunk.export(chunk_name, format="ogg") 


In [ ]:
# Lets exstract the features from the test audio
import librosa
import numpy as np

def features_extractor(file):
    #load the file (audio)
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    #we extract mfcc
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    #in order to find out scaled feature we do mean of transpose of value
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features


import os
extracted_features_=[]
for i in range(0,12):
    file_name = os.path.join(os.path.abspath("./"),str(i) + ".ogg")
    data=features_extractor(file_name)
    extracted_features_.append([data])

In [ ]:
import pandas as pd
extracted_features_df_=pd.DataFrame(extracted_features_,columns=['feature'])


In [ ]:
X_=np.array(extracted_features_df_['feature'].tolist())

In [ ]:
#We use the same features than before
X_prueba = X_
X_prueb = X_prueba.tolist()
X_pru = []
X_pri = []

for i in range(len(X_)):
    for j in range(0,40):
        if j==6 or j==8 or j==12 or j==16 or j==24 or j==36 or j==2:
            X_pru.append(X_prueb[i][j])
    X_pri.append(X_pru)
    X_pru = []
            
                
X_pro = np.array([np.array(xi) for xi in X_pri])

In [ ]:
with open('../input/birdclef-2022/scored_birds.json') as sbfile:
    scored_birds = json.load(sbfile)

In [ ]:
predict__=model.predict(X_pro)

In [ ]:
# This is where we will store our results
pred = {'row_id': [], 'target': []}

# Process audio files and make predictions
for afile in file_list:
    
    # Complete file path
    path = test_audio_dir + afile + '.ogg'
    
    # Open file with librosa and split signal into 5-second chunks
    # sig, rate = librosa.load(path)
    # ...
    
    # Let's assume we have a list of 12 audio chunks (1min / 5s == 12 segments)
    chunks = [[] for i in range(12)]
    
    # Make prediction for each chunk
    # Each scored bird gets a random value in our case
    # since we don't actually have a model
    for i in range(len(chunks)):        
        chunk_end_time = (i + 1) * 5
        j=0
        for bird in scored_birds:
            
            # This is our random prediction score for this bird
            score = predict__[i][j]
            maximun = max(predict__[i])
            j=j+1
            # Assemble the row_id which we need to do for each scored bird
            row_id = afile + '_' + bird + '_' + str(chunk_end_time)
            
            # Put the result into our prediction dict and
            # apply a "confidence" threshold of 0.5
            pred['row_id'].append(row_id)
            pred['target'].append(True if score > 0.008 else False)

In [ ]:
results = pd.DataFrame(pred, columns = ['row_id', 'target'])

# Quick sanity check
print(results.head()) 
    
# Convert our results to csv
results.to_csv("submission.csv", index=False)